# LangChain

[LangChain](https://python.langchain.com/en/latest/index.html) is an open-source framework for developing apps powered by Large Language Models (LLMs) such as [ChatGPT](https://openai.com/blog/chatgpt). With a few lines of code, you can build apps that answer questions from documents, databases, and more. The following example uses LangChain to "chunk" Microsoft's 2022 annual report (a 90-page Microsoft Word document) and ChatGPT to answer questions from the report. In order to run this notebook, you must store your OpenAI API key in an environment variable named `OPENAI_API_KEY`.

![](Images/langchain.png)

Begin by using LangChain's loader for DOCX files to chunk Microsoft's 2022 annual report and create a vector database from the chunks. By default, LangChain uses OpenAI's [Embeddings API](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings) (`text-embedding-ada-002`) to create vectors from chunks of text, and it stores the vectors in an in-memory database provided by [DuckDB](https://duckdb.org/). This example persists the vector database in the "Data" subdirectory so the database can easily be recreated later.

In [1]:
from langchain.document_loaders import Docx2txtLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

# Load the annual report
loader = Docx2txtLoader('Data/annual-report.docx')

# Create an index over it using an in-memory vector store
index = VectorstoreIndexCreator(vectorstore_kwargs={ 'persist_directory': 'Data'}).from_loaders([loader])

Using embedded DuckDB with persistence: data will be stored in: Data


Now that the annual report is chunked and vectorized, asking a question is a simple matter of calling the `query` method of the `VectorstoreIndexWrapper` object in `index`.

In [2]:
# Perform a query on the annual report
index.query('Did Microsoft repurchase shares in 2022, and if so, how many shares did it repurchase and what was the value of those shares?')

' Yes, Microsoft repurchased 95 million shares in 2022, with a total value of $28.033 billion.'

Of course, you don't want to have to chunk and vectorize a document or set of documents each time an app starts up. The next example demonstrates how to recreate the vector store from the files persisted in the "Data" subdirectory. Currently, this requires adding a method to the `VectorstoreIndexCreator` class:

In [3]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# Add a from_persistent_index method to VectorstoreIndexCreator
def from_persistent_index(self, path: str)-> VectorStoreIndexWrapper:
    vectorstore = self.vectorstore_cls(persist_directory=path, embedding_function=self.embedding)
    return VectorStoreIndexWrapper(vectorstore=vectorstore)

VectorstoreIndexCreator.from_persistent_index=from_persistent_index

The `from_persistent_index` method can now be used to recreate the vector store and ask questions:

In [5]:
index = VectorstoreIndexCreator().from_persistent_index('Data')

index.query('What was Microsoft\'s year-over-year increase in cloud revenue from 2021 to 2022?')

Using embedded DuckDB with persistence: data will be stored in: Data


' 32%'

The answer to the question above can be gleaned from page 83 of the annual report. Did the LLM get it right?